# Applications 1: Unemployment Flows and Search

This notebook explains step by step the solutions and method of *Quantitative Economics* of the chapters concerning *Linear Algebra*, *A Lake Model of Employment and Unemployment* and *Search with Offer Distribution Unknown*.

The aim of the day is to show you a cool application of what you have seen so far in the course, introduce you to some useful tools to code and solve models (Value Function Interation and simulation), and have some *fun* playing around with the models.

This is **not** intendend to be a throughfully explained guide, but rather a *companion* to the text and a way to save you some time coding. 

**Keep the book page open!**

### Summary:
1. Quick Linear Algebra in Python
2. The Bathtub (Lake) Model
3. McCall Search
4. Offer Distribution Unknown
5. Wrap up

In [ ]:
# Exercute this cell to import the libraries we are going to use!
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## 1. Quick Linear Algebra in Python

### 1.1 NumPy arrays as vectors

### 1.2 Matrices and linear algebra

### 1.3 Matrices as Maps

### 1.4 Eigenvalues and Eigenvectors with NumPy

## 2. The Bathtub (Lake) Model

### 2.1 The idea

You may have been introduced to the treatment of labour markets as a rather *static* entity - think about the standard Mortensen-Pissarides model - devoted to solve for steady-states and do comparative statics exercises.

A more advanced and modern view of labour markets sees them as dynamic entities, and studies the *flows* in and out of unemployment. For example: how many people lose their job in a period of time, how many people find a job and how these quantities evolve through time. This is (in a very simplistic version) what we are going to do here.

The idea is that unemployment is like a Bathtub (or a Lake): 

*picture*

- Every period, a fraction $\alpha$ of employed workers enters trought the tap into the bathtub (loses their jobs), a fraction $\lambda$ un unemployed workers drains down the bathtub (finds a job).

- Simultaneously, some water leaks out (people ~~die~~ retire at rate $d$) from both the bathtub and the deposit (from unemployed and employed workers) and we add some more to compensate for that to the bathtub ($b$, newborns entry to the labour force into unemployment).

If you have ever taken a bath, you should know that if you leave the tap open and drain at the same time you can get an steady level of water - with in our case is unemployment - through any point in time.

What we are going to do in the simulation is to leave the tap open (how much is our $\alpha$) and putting a filter in the drain pipe ($\lambda$) and watch as the water reaches a constant level.

We are going to name three stocks:

- $E_t$: Stock of employed workers - volume of water down the deposit.
- $U_t$: Stock of unemployed workers - volume of water in the bath.
- $N_t$: Labour force stock - volume of water in all the system (in the bath plus in the deposit).

And two rates:
- $u_t$: unemployment rate - % of water in the bath
- $e_t$: employment rate - % of water in the deposit

You can see that it is going to be easier to think in rates rather than stocks.

### 2.2 Matrix equations and Law of Motion

In a more formal way, we can resume all of our system with the following equations:

$$ E_{t+1} = (1-d)(1-\alpha)E_t+(1-d)\lambda U_t $$

$$ U_{t+1} = (1-d)\alpha E_t+(1-d)(1-\lambda)U_t+b(E_t+U_t) $$

This is for stocks, and for rates just divide by $N_t$:

$$ e_{t+1} = (1-d)(1-\alpha)e_t+(1-d)\lambda u_t $$

$$ u_{t+1} = (1-d)\alpha e_t+(1-d)(1-\lambda)u_t+b $$

Whete $x_t = X_t/N_t$

A more compact way of expressing these equations is encapsulating them in matrix form: 

$$\begin{bmatrix}E_{t+1}\\U_{t+1}\end{bmatrix} = \begin{bmatrix}(1-d)(1-\alpha)& (1-d)\lambda \\(1-d)\alpha & (1-d)(1-\lambda)+b \end{bmatrix}\begin{bmatrix}E_{t}\\U_{t}\end{bmatrix}$$

$$ X_{t+1} = A X_{t} $$

For stocks where $A$ is what we call *Transition Matrix* and  $X_t$ is a vector of stocked stocks. In rates:

$$ x_{t+1} = \hat{A} x_{t} $$

Where $\hat{A}$ is the transition matrix for rates and $x_t$ is a vector of stocked rates.

This is what we just did with systems of equations above in liner algebra! And the reasons are similar: it allows us to solve it quickly and elegantly, and introduces us to the concept of ***Law of Motion***.

We call a ***Law of Motion*** to the equations that encapsulate the way that stock variables evolve trough time. This is easier to see in matrix form: 

                        Stocks tomorrow = Transformation  matrix * Stocks today

The *Transition Matrix* is then **a map**, mapping the present (stocks) into the future (stocks). So a ***Law of Motion*** is simply a mapping through time.


### 2.3 Finding the Steady State

We were interested in finding a **Steady-State** level of water in the bathtub, so that water today is the same as water tomorrow. This translates into solving for $X_t = X_{t+1}$ in the equations before.

If $b>d$ then our stocks are constantly growing, so the volume of water is always increasing: it is never going to be the same at two points in time - just because is ever-ingresing.

But if we divide by $N_t$, and $N_t$ itself is growing (at a rate $g = b-d$), then the rates system is going to always sum up to 1. So there can be a point in time where the % of water in and out of the bathtub is the same.

More formally, we can find a ***fixed point*** $\bar{x}$ such that

$$ \bar{x} = \hat{A}\bar{x} $$

Looks familiar? It should, because the solution is that $\bar{x}$ is an *eigenvector of $\hat{A}$ associated with a unit eigenvalue.*

See? All that boring math comes to the rescue.

**Exercise 2.1** Find the steady-state of an economy with parameters {$\alpha$: 0.01, $\lambda$: 0.25, $b$:0.01, $d$:0.005} using linear algebra.

* _Tip 1: remember that numpy normalizes eigenvectors automatically_
* _Tip 2: remember that $e_t + u_t = 1$_

In [ ]:
# Insert code here!

Another way of finding the steady-state rates is through **iteration**: under some circumstances that are fulfilled here (mainly $\hat{A}$ being a *contraction mapping*) from any starting $u_t$ and $e_t$ and transition matrix $\hat{A}$  if we just *simulate* the economy (or *iterate* by using the implied $x_{t+1}$ as your new $x_t$, etc) for enough periods, you'll see the rates converge to their steady-state values.

**Exercise 2.2** Find the steady-state of an economy with parameters {$\alpha$: 0.01, $\lambda$: 0.25, $b$:0.01, $d$:0.005} using iteration starting from everyone being unemployed. How many iterations do you have to go through?

In [ ]:
# Insert code here!

**Exercise 2.3** Modify your code above, adding two variables `u_path` and `e_path` such that the following code plots the paths of $u_t$ and $e_t$ :

In [ ]:
plt.plot(u_path, label="unemployment") # plot the path of unemployment and label it "unemployment"
plt.plot(e_path, label="employment") # plot the path of employment and label it "employment"
plt.xlim(0,len(u_path)-1) # set the limits to the x axis (start, finish)
plt.ylabel("%") # label the y axis
plt.xlabel("iterations") # label the x axis
plt.legend(loc='best', fontsize=14) # show the legend
plt.show() # show the plot

**Cool Exercise** You may have notice that all of this is very nice and smooth. But if you have ever seen **real** plots of time series umeployment rates they look way less cut clean. One reason is that in the real world $\alpha$ and $\lambda$ are not constant, but change through time.

Your job is to create a plot of unemployment that reflects booms and busts by recycling your code above. 

a) Suppose we start from the steady state above and then **every 10 periods** we switch from *good state* to a *bad state* such that {$\alpha$:0.02, $\lambda$:0.20}. Morevover, every 60 periods, we have a *very bad* state {$\alpha$:0.03, $\lambda$:0.15}. Plot the path of unemployment for 100 periods. Does this look more realistic that the exercise before? How?

b) A way to make our plot more realistic is introducing lags into the job finding rates, so $\lambda$ recovers from a shock only slowly. That is, we allow $\alpha$ to "jump", but not $\lambda$. In particular assume lambda changes according to:
$$ \lambda_t = \lambda_{t-1} + \rho(\bar{\lambda} - \lambda_{t-1})$$

That is, each period, $\lambda_t$ only recovers a fraction $\rho$ from its distance to the steady-state. You can check that only when $\lambda_t$ is in steady-state, $\lambda_{t+1}=\lambda_t$.

Repeat the exercise before, but incorporating this and see how the plot changes. Play with different $\rho$s - for example, in part a) $\rho=1$

In [ ]:
# Insert code here! (or do it somewhere else)

The book now goes over simulating the process for a single agent using some properties of Markov chains, but we are going to skip this for the time being. By all means read it at home and try to replicate their fancy plot.
______________

## 3. McCall Search

### 3.1 The idea

If you do the *cool exercise* from last section, you should realise that exogenous job finding/job separation rates are pretty boring, even when we allow for some time dependance.

In this section we endogenize the job finding rate, by assuming that when our unemployed workers can choose to reject a job offer, because now jobs have different wages, depending on the distributions of wages they can possible receive. This is (the return of) the **McCall model**.

In a way, this is going into some micro fundations of the Bathtub model, where in principle we were ignoring that workers are individuals whose choices influence the outcome of the labour market. And to some extent, we were also ignoring the fact that not all jobs are the same.

From here we will follow the book for the formulation of the model.

_**Note** The notation is pretty tricky in the book, as you should have notice. We'll try to navigate it with care._

_**Note for current SGPE** the notation here is slightly different from our tutorials._

### 3.2 Notes on Dynamic Programing

If you are a SGPE student, you should be familiar with most of terms we use here, in particular:
* Value function
* Policy function
* Bellman equation
* And althought not mention in the book, also State Variable, Choice Variable and Parameters

**Quick exercise ** What is the difference between a state variable and a choice variable? And between a parameter and state variable? And between a Value function and a Bellman equation?

Solving for the true Value function - the one that has the unknown optimal policy inside - is not trivial with pen and paper, but computers can give us a hand on this! 

The easiest procedure that we do here is called "Value Function iteration", and the principle it works on is simple: 
* We make an initial guess and try our best today. 
* Tomorrow, we take today's choice (the best we could do with our initial guess) as our guess and again try to do our best. Probably we will be able to choose something different, something better.
* Repeat until turns out that the best we can do one day is *exactly* what we did yesterday. 

This means that it can't be improved upon, and thus we have find the optimal plan. It is optimal because there is no improvement we can make on it.

There is of course some mathematical conditions all of this has to meet. But a nice result is: ***if you can encapsulate the problem in a set of Bellman equations, then you will find the optimun by iterating.***

You'll learn more of how this magic works on Monday.

So the variables of interest are:
- Vector of our choice variable - which in this case in binary: $1$ means accept the wage offer, $0$ reject: $C = [C_1,C_2,...C_s] : C_i\epsilon \{1,0\}$ for each wage $w_,w_2,...,w_s$.


- Vector of value functions $V = [V_1,V_2,...V_s]$ which contains the optimal flow values associated with our choice vector. 

    That is, if we prefer being unemployed to accepting wage $w_1$, then $C_1 = 0$ and $V_1=U$. If we prefer accepting wage $w_2$ to being unemployed, then $C_2 = 1$ and $V_2=V(w_2)$.

Our Bellman are then:

$$V^{t+1}_s = u(w_s)+\beta [(1-\alpha)V^{t}_s +\alpha U^{t} ]$$

$$U^{t+1} = u(c)+\beta(1-\gamma)U^{t}+\beta\gamma\sum_s{\pi_s\max\{V^{t}_s,U^{t}\} }$$ 

Notice that I ahve added a $\pi_s$ that is missing from the book.

**Quick exercise ** Why do we need the $\pi_s$?

**Quick exercise ** Why does the flow value $U$ don't have an $s$ subscript? Is $U$ a vector or a scalar?

** Quick exercise ** If we wanted to make this a MP model, what would $\gamma$ have to represent?

**\* Some deatils in case you are interested **

Calling $\hat{V}^{t}$ to the flow utility values associated with our choices at iteration $t$, our next guess$\hat{V}^{t+1}$ is going to be:

$$U^{t+1} = \frac{ u(c)+\beta\gamma\sum_s{\pi_s*{\hat{V}_s^{t}} }}{1-\beta(1-\gamma)}$$ 

$$V^{t+1}_s = u(w_s)+\beta [(1-\alpha)\hat{V}^{t}_s +\alpha U^{t+1} ]$$

$$\hat{V}^{t+1}=\max\{U^{t+1},V^{t+1}\}$$

Again, sorry if the notation is confusing!

### 3.2 Adding government policies

The book adds the goverment to set the tax that balances the budget - and the unemployment benefit $c$ has to come from somewhere! This allows the book to go on an explore welfare maximizing taxes and benefits.

We are not as interested on this as in showing you how, once we have solved for agent's optimal choice now we can find the optimal unemployment using the Lake (Bathtub) methodology we just saw. Knowing who is going to be employed and unemployed we can find then the taxes that balance the budget, and play around with the combination of $(c,T)$ that maximizes everyone's utility.

Beyond the interesting political economy question this poses, it allows us to see what happens when you have a problem inside of a problem - something very usual and cool that happens to real-life researchers.

In this case, we need to find the optimal policy $C^*$ for the worker for each unemployment subsidy first, so we can then calculate the associated steady-state unemployment rate $u^*(c,T)$ and use it to balance the budget so we get the new optimal unemployment rate $u^*(c,T^*)$. Then we can find the optimal $c^*$ that maximizes everyone's welfare. At the end, we get the unemployment rate $u^*(c^*(T^*))$

### 3.3 Solving the model: Code walkthrough

Let's walk through the code to understand how it works.

First, note that we have two **classes**: `LakeModel_Equilibrium(object)`, `LakeModel` and `LakeModelAgent`. Of these we are going to ignore the last one, as we skip over that part before.

`LakeModel` contains code that finds automatically, for some aprameters, the steady-state levels of employmet and unemployment. It also has two functions to simulate the paths of stocks and rates as they converge to their steady-state levels - just what we did ourselves before!

`LakeModel_Equilibrium` contains functions to solve for the equilibrium of the MacCall model for a set of parameters we feed in it:


- `__init__(params)` stores the parameters we input and creates an *instance* with them.


   - `U(c)` is simply a CRRA utility function. Enter a consumption (c) and returns utility. It says that it works for vectors and scalars, but actually we have to modify it a bit to allow it work for scalars too.
   
   
   - `solveMcCallModel(c,w,eps = 1e-6)`: Given an unemployment benefit $c$ and a vector of wages $w$ finds the **optimal choice** of the agent, $C^{*}$, which is a vector with the same length as the  vector of wages with 1s and 0s (1 if the agent accepts that wage and 0 if he rejects. It also gives us back the maximized set **Value functions** (how much flow utility do we get for each wage). If does so iterating from the initial guess of **rejecting every wage**: $C_0=[0,0,..0]$ and $V_0 = [0,0,...0]$ using the next function. The iteration stops when the difference between our initial guess for the value function and the one resulting from the iteration are *close enough* (the paramenter `eps` define what  *close enough* is).
   
   
   - `iterateValueFunction(c,w,V)` given an unemployment benefit $c$, wage vector $w$ and initial guesses for value functions $V$, calculates the flow value of unemployment $U$ and being employed with each of the wages $V_s$ in $w$. Then *stacks them*, compares each flow value $V_s$ with $U$, and chooses the highest value for each case. **This will be our next guess in the solveMcCallModel() function**.
   
   
   - `solve_for_steadystate(c,T)` finds the equilibrium employment and unemployment rates and total welfare, given a uemployment benefit $c$ and a tax $T$. It does do by invoking `solveMcCallModel` and a function of `LakeModel` that finds the equilibrium unemployment rate. Remember that this time the job finding rate is *endogenous* and depends in the wages that are floating around in the economy - and agent's best choices.
   
   
   - `find_steady_state_tax(c)` is the last function: finds the optimal $T$ for the unemployment benefit $c$ we choose, by calling `solve_for_steadystate(c)` and finding the T that balances the goverment's budget.




Recap:
* Formulate the inital choice of the worker as ***reject all offers***, find the corresponding flow values for each wage, and then choice between them and the flow value of unemployment. Store our **choice vector** and **chosen flow values**.
* Take the last **chosen flow values** as our initial guess and repeat step 1 until the resulting chosen values are the same as the initial values. Store this **_optimal_ choice vector** and **_optimal_ flow values**.
* With the **_optimal_ choice vector** (or optimal policy) we can now calculate the job finding rate of the economy! Find the **equilibrium unemployment rate** for the choice of unemployment benefit $c$ and tax $T$, $u(c,T)$.
* Once we know $u(c,T)$, find the tax $T^*$ that ensures a balanced budget, and so the corresponding **equilibrium unemployment rate** $u(c,T^*)$. 

***Note to current SGPE*** Notice that we haven't formulated the *reservation wage* equation. We don't need it! The computer can solve for numerical values without it using Value Function Iteration.

Now let's play around with the code!

** Exercise 3.1** Below I copy-pasted the code for the utility function. I tried to use it with scalars, but it doesn't work. Can you repair it so it does?

In [ ]:
def U(c, sigma=3.0):
    r'''
    Utility function of the agent        
    Parameters
    -------------        
    c - (array or float) consumption of the agent
    sigma - (float) risk aversion parameter  
    Returns
    ----------        
    U - (array or float) Utility of the agent for each level of consumption
    '''
    negative_c = c < 0.
    if sigma == 1.:
        U =  np.log(c)
    else:
        U= (c**(1-sigma) - 1)/(1-sigma)
    U[negative_c] = -9999999.
    return U

U(9.0) # Executing this cell should work fine and now throw any errors!

**Exercise 3.2** In the cell below I copied the `solveMcCallModel` and `iterateValueFunction` functions in a stand-alone version. Just with these two we can get a nice plot showing us how our Value function guesses converge to the true one.

There is going to be three kinds of jobs: *bad* (2.0), *good* (3.0), and *awesome*(5.0). The unemployment benefit $c$ is fixed at 1.0. Therefore we have a vector of length 3 for each wage called `wages` and the probabilities associated with them are in a vector called `p`. The rest of parameters are detailed below.

Add some bits to the functions below such that the next cell plot work!

_**Note** it is necessary that you execute the cell abovethat contains the stand alone Utility function for it to work!_

In [ ]:
# Defining some parameters
alpha =0.012 # Job destruction rate (exogenous)
gamma = 0.5 # Offer arrival rate (exogenous)
beta = 0.95 # Discount rate 
sigma = 3.0 # Risk aversion (matches the default above)
wages = np.array([2.0,3.0,5.0]) # Wage vector
p = np.array([0.3,0.5,0.2]) # Probability of receiveing each wage (adds to 1)

def iterateValueFunction(c,w,V): 
    r'''
    Iterates McCall search value function v    
    Parameters
    ----------    
    c - (float) Level of unemployment insurance
    w - (len(3) 1d-array) Vector of possible wages
    V - (len(3) 1d-array) continuation value function if offered w[s] next period        
    
    Returns
    --------
    V_new - (len(3) 1d-array) current value function if offered w[s] this period 
    Choice - (len(3) 1d-array) do we accept or reject wage w[s]
    '''
    S = len(p)
    Q = p.dot(V)# value before wage offer
    V_U = (U(c*np.ones(S)) + beta*gamma*Q)/( 1-beta*(1-gamma) )
    #stack value of accepting and rejecting offer on top of each other
    stacked_values = np.vstack([ V_U,
                                U(w) + (1-alpha)*beta*V + alpha*beta*V_U ]) 
                                    
    #find whether it is optimal to accept or reject offer    
    V_new = np.amax(stacked_values, axis = 0) 
    Choice = np.argmax(stacked_values, axis = 0)
    return V_new,Choice
        
def solveMcCallModel(c,w,eps = 1e-6):
    r'''
    Solves the infinite horizon McCall search model        
    Parameters
    -----------        
    c - (float) Level of unemployment insurance
    w - (len(3) 1d-array) Vector of possible wages         
    eps - (float) convergence criterion for infinite horizon
    
    Returns
    --------        
    V - (1d-array) Value function that solves the infinite horizon problem        
    Choice - (1d-array) Optimal policy of workers
    '''
    S = len(p)
    v = np.zeros(S) #intialize with zero
    diff = 1.0  #holds difference v_{t+1}-v_t
    # Some code could go here
    # and here
    while diff > eps:
        v_new,choice = iterateValueFunction(c,w,v) 
        # Some code could go here
        # and here
        diff = float(np.amax( np.abs(v_new-v) ))#compute difference between value
        v = v_new #copy v_new into v #add in infinte horizon solution
            
    return v,choice # maybe you could add something here?    

In [ ]:
# This cell should work fine and plot something nice!
v_star, c_star, vs, cs = solveMcCallModel(1.0,problem_1.wstar)
labels = ["$V_1^*$","$V_2^*$","$V_3^*$"]
colors = ["b","g","r"]

plt.figure(figsize=(7,5))
plt.plot(vs)
for s in range(len(v_star)):
    plt.axhline(v_star[s], ls='--', lw=0.9,color=colors[s], label=labels[s])
plt.legend(loc='best')
plt.xlabel("iterations")
plt.ylabel("flow utility")
plt.title("Value Function Interation")
plt.show()

** Exercise 3.3** How many iterations do you needed to converge? Does the worker accept all the three wage offers? Modify the code above to get the `solveMcCallModel` fucntion to print this information. 

** Cool Exercise ** You can use the code above to have a lot of fun. For example:
- What happens if you change the wage offers probability distribution? For example, make $p=[0.6,0.35,0.05]$
- What happens if you change the wages available? For exmaple, add a *terrible* wage, or an extremaly good one: $w = [1.0,2.0,3.0,10.0]$ and $p=[0.1,0.45,0.4,0.05]$
- What happens if you set the unemployment benefit to be 0? Will the worker accept a wage of 5.0 if the unemployment benefit is 3.0?
- What happens if the worker gets more impatient?

In [ ]:
# Insert code here! (or do it somewhere else)

** Exercise 3.4 ** The cell bellow imports all the `LakeModel` code. Can you call the right function to get the equilibrium unemployment rate related to the parameters we jsut used above? What is the tax that the goverment has to set to get 1.0 to all the unemployed workers?

In [ ]:
import LakeModel.py

# Defining wages distribution
w_1 = np.array([2.0,3.0,5.0])
pdf_1 = np.array([0.3,0.5,0.2])

# Your code here!

** Exercise 3.5 ** Find all of the above but for the case of the extended wage offers and change in probabilities $w = [1.0,2.0,3.0,10.0]$ and $p=[0.1,0.45,0.4,0.05]$.

In [ ]:
# Your code here! - no need to import code again

** Cool Exercise ** Iterate for certain reasonable values of $c$  and find (roughly) the optimal unemployment benefit $c^*$. If our initial 1.0 too high or too low? Is a policy of 0 unemployment benefit the best one?

** Cool Exercise ** Find the optimal tax if only employed workers pay the tax. Is this arrangement welfare improving for everyone?

** Cool Exercise ** The `LakeModel` class can simulate the convergence to steady-state, but for fixed parameters $\gamma$ and $\alpha$. Change the code or write your own to simulate the response onf the unemployment rate to a shock such that $\alpha = 0.02$ after 10 periods. _**Tip:** Use the functions from LakeModel.py to solve for the implied steady-states and their associated job finding rates, and then recycle the code in section 2 for the plot._

In [ ]:
# Insert code here! (or do it somewhere else)

--------------
After these exercises I hope you can appreciate the beauty of Object Oriented programming. How nice it is to only have to import the code and then call the functions without the user having to know all the bits inside?

If you are writting code, organizing stuff in classes and fucntions helps you a lot to get your head around the code and can potetially help you understand the theory better.

And if you dind't write the code but have to go inside it to change stuff, it is much more easy to understand what is going on and which function depends on which if you compartiment them into classes. Just comapte the `LakeModel` bit with our own code in section 2. Isn't the *classified* one nicer?

## 4. Offer Distribution Unknown

### 4.1 Changes to the previous Model

Now we go beyond to explore the case were the agent does not know, a priori, the distrbiution of wages. Note that before the agen calculates the best policy instantaneously - actually, ALL the agents calcualte the best response at the begining of the period. The last **cool exercise** above deal precisely with this assumption and what it involved for unemployment dynamics.

In this world the agen't does not know a priori the distribution, and learns about it by sampling wage offers. For exmaple, if the agent receives an offer of 3.0, 3.0, 2.0, 3.0, then at time $t=4$ she thinks that the distribution is $E(p)_{t=4}=[0.25,0.75,0.0]$. But if then gets a wage offer of 5.0, then she *updates har beliefs* to $E(p)_{t=5}=[0.2, 0.6, 0.2]$.

However we are going to suposse that the agent is not *that* ignorant about the worl, and she just doubts between two continuous distributions: $G$ and $F$ - you can think about them as "Good times " and "Bad Times". In "Bad Times" you have more chances of getting a low wage offer that in "Good Times". She updates her beliefs about the right distribution using  the *Bayes Rule*, which is a weighted average of how likely the wage equence $w_t$ is under each of the possible distributions. More formally, call $\pi_t$ the probability that we are in $f$ (bad times) and $w_t$ the sequence of wages obtained at time $t$:

$$ \pi_{t+1}=\frac{\pi_t f(w_{t+1})}{\pi_t f(w_{t+1})+(1-\pi_t) g(w_{t+1})} $$

This is very interesting, because is introducing a bit sof something called ***learning***: the agent does not have all the information about the *state of nature*, but she can *learn* which one is the true one. Learning takes time, and it is only perfect in the limit (as $t$ goes to infinity the agent *must* know the true distribution).

Also note that now in the Value Function equation in the book we don't take utilities any more.

** Quick exercise ** What does this imply to our agent risk aversion?

** Quick exercise ** Why there are integrals now? Where were they in section 3?

### 4.2 Value Function Iteration (VFI) vs Reservation Wage Operator

You already know the mechanics behind the VFI: give a guess to $\hat{V}$, iterate (calculate the new $V_s$ and $V_u$, compare them, and choose the highest ones to be our new $\hat{V}$) until convergence. The fact that the equations can be encapsulated into a Bellman equation ensures convergence. Here the mecahnichs are similar, but notice that now we have a *continous* distribution, so we have potentially **infinite** $V_s$. This only complicates the math a little, but don't worry, the code will do it for us!

Also before we didn't calculate the ***reservation wage*** (the wage for which we should reject all offers below it and accept above), because getting the choice vector was realatively easy - our wage distribution had only three points. Now the choice vector can have **a lot more** wages, so it makes sense to us to reformulate the Choice vector $C$ as $\bar{w}(C)$ being $\bar{w}$ our reservation wage.

It is therefore a more clever approach, if we *know* that for some wage the agent *must* be indifferent between accpeting and staying unemployed ($V(\bar{w})=V_u$) to just use the functional equation of the reservation wage (equation 4 in quantecon notes). 

This is more clever because now we onyl care about one number: the reservation wage. For VFI we needed to calculate and compare (maximize) **all** possible wages. Here we only need to calculate a single "Value function". This makes the algorithm much faster.

At the core we are still going to use iteration. We know we will converge because the *Reservation Wage* function is a **contraction mapping**.

Remember maps for part 1? We are searching for a **fixed point** (so $f(x)=x$), a contraction map (call ti $M$) reduces the distance between $x$ and $f(x)$ such that $M(f(x)-x)<(f(x)-x)$.

We want to bring the distance between $f(x)$ and $x$ down to 0 (or as close to 0 as we can), so if we apply the map again: $M(M((f(x)-x))<M(f(x)-x)<(f(x)-x)$ we get then closer. Apply the Map enough times and you will bring the distance down to (or very close to) zero!

You can read the [Take 2](http://quant-econ.net/py/odu.html#id12) part of the notes up for the technical details.

** Quick question ** After looking at the equations of Take2, what is our mapping function in this context?

### 4.3 Code Walktrhough
* Code walkthrough.
* Understanding the pictures.



### 4.4 Playing with the code
* Exercise 1 in the book
* The extension by Sargent

## 5. Wrap up